1. 문서내용 read
2. 문서 토큰화 (쪼갬)
  - 토큰수 초과로 답변 생성하지 못할수 있다.
  - 문서가 길면(input) 답변 생성이 오래걸린다.

3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을때, 벡터 데이터베이스에 유사도 검사
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [1]:
%pip install python-dotenv langchain langchain-upstage langchain-community langchain-text-splitters

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.2/374.2 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9

In [3]:
!pip install docx2txt

  Preparing metadata (setup.py) ... done
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3959 sha256=ba9c1d0d2a1879e65de59f7dc8904fdc27b01be053f3eef28f43213f57afd427
  Stored in directory: /root/.cache/pip/wheels/22/58/cf/093d0a6c3ecfdfc5f6ddd5524043b88e59a9a199cb02352966
Successfully built docx2txt


In [4]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import docx2txt

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap = 200,
    )
loader = Docx2txtLoader('/content/ESG.docx')

document_list = loader.load_and_split(text_splitter=text_splitter)

In [5]:
document_list

[Document(metadata={'source': '/content/ESG.docx'}, page_content='Cleaner\xa0Ocean\n\nfor\xa0Happier\xa0Earth\n\n해양환경공단\xa02022\xa0ESG경영보고서\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n깨끗한\xa0바다\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n상생의\xa0바다\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n신뢰의\xa0바다\n\n\n\n\n\n\n\nAbout\xa0this\xa0RepoRt\n\n\n\n\n\n\n\n\n\n보고서\xa0개요\n\n해양환경공단은\xa0ESG\xa0부문\xa0주요\xa0추진활동과\xa0성과를\xa0내·외부\xa0이해관계자들에게\xa0알리고,\xa0보다\xa0많은\xa0이해관계자들의\xa0기대와\xa0요구에\xa0귀\xa0기울이기\xa0위해\n\n2012년\xa0첫\xa0보고서\xa0발간\xa0이래\xa0매년\xa0ESG경영보고서를\xa0발행하고\xa0있습니다.\n\n이번\xa0보고서는\xa0ESG경영을\xa0포함한\xa0비재무적\xa0성과와\xa0재무적\xa0성과를\xa0한\xa0권에\xa0통합하여\xa0보고함으로써\xa0환경,\xa0사회,\xa0지배구조\xa0성과와\xa0공단\xa0이해관계자가\n\n해양환경공단의\xa0경영\xa0현황과\xa0기업\xa0가치를\xa0보다\xa0분명하게\xa0파악할\xa0수\xa0있도록\xa0기획하였습니다.\n\n\n\n\n\n보고\xa0원칙\n\n본\xa0보고서는\xa0ESG경영\xa0성과를\xa0보다\xa0투명하게\xa0공개하기\xa0위하여\xa0국제\xa0지속가능경영보고\xa0지침인\xa0GRI(Global\xa0Reporting\xa0Initiative)\xa0Standards의\xa0핵심적\n\n부합\xa0기준(Core\xa0Option)을\xa0준수하였으며,\xa0유엔글로벌\xa0콤팩트\xa0원칙과\xa0ISO\xa0260

In [7]:
import os
from google.colab import userdata

os.environ["UPSTAGE_API_KEY"] = userdata.get('Upstage_api')

In [8]:
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings

load_dotenv()

embedding = UpstageEmbeddings(model = 'solar-embedding-1-large')

In [9]:
%pip install langchain-chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 9.3 MB/s eta 0:00:00


In [10]:
from langchain_chroma import Chroma
from chromadb.utils import embedding_functions

database = Chroma.from_documents(
    documents = document_list,
    embedding = embedding,
    collection_name = 'Chroma-ESG',
    persist_directory = '/content/chroma'
)

In [11]:
query = 'KOEM ESG 경영체계가 어떻게 돼?'
retrieved_docs = database.similarity_search(query, k=3)

In [12]:
retrieved_docs

[Document(metadata={'source': '/content/ESG.docx'}, page_content='고객만족도\n\n중대재해\n\n우수\n\n\n\n\n공정경제\xa0\xa0\xa0\xa0\xa0\xa0\xa0공익우선\xa0\xa0\xa0\xa0\xa0\xa0\xa0국민참여\xa0\xa0\xa0\xa0\xa0\xa0일하는\xa0보람\n\n\n\n핵심가치\n\n\n\n\n\n\n\n건강한\n\n해양생태계\xa0보전\n\n\n안전한\n\n해양환경\xa0조성\n\n\n국민의\n\n해양\xa0이용·향유\xa0증진\n\n\n사람중심\n\n안전·혁신\xa0경영\n\n\n\n\n\n\n\n전략목표\n\n\n\n및\xa0과제\n\n\n1.\xa0해양환경\xa0탄소중립\xa0기반\xa0확대\n\n2.\xa0해양폐기물\xa0전주기\xa0관리\xa0강화\n\n3.\xa0해양환경\xa0조사연구\xa0플랫폼\xa0구축\n\n\n4.\xa0SMART\xa0해양사고\xa0대응역량\xa0강화\n\n5.\xa0협력적\xa0해양오염\xa0예방·대응\xa0체계화\n\n6.\xa0해양사업\xa0효율성\xa0강화\n\n\n7.\xa0해양환경\xa0이용·향유\xa0자원화\xa0확대\n\n8.\xa0지속성장\xa0미래사업\xa0육성\n\n9.\xa0해양환경\xa0대국민\xa0서비스\xa0확대\n\n\n10.\xa0효율성\xa0중심의\xa0경영혁신\xa0실현\n\n11.\xa0건강하고\xa0안전한\xa0근로환경\xa0제공\n\n12.\xa0투명하고\xa0공정한\xa0조직문화\xa0조성\n\n\n전문역량\xa0\xa0\xa0\xa0\xa0\xa0\xa0안전우선\xa0\xa0\xa0\xa0\xa0\xa0\xa0열린혁신\xa0\xa0\xa0\xa0\xa0\xa0\xa0협력성장\n\n\n\n\t\t\t\t해양의보전과이용을선도하는해양환경국민플랫폼기관\tE\xa0깨끗한\xa0바다\tS\xa0상생의\xa0바다\tG\xa0신뢰의\xa0바다\tAPPENDIX\n\n\n2021/2022\xa0해양환경공

In [13]:
from langchain_upstage import ChatUpstage
llm = ChatUpstage()

In [14]:
query = '중대성 평가의 개요에 대해서 설명해줘'
retrieved_docs = database.similarity_search(query, k=3)

In [15]:
prompt = f'''[Idendtity]
- 당신은 해양환경공단의 경영자야.
- [Context]를 참고해서 사용자의 질문에 답변해주세요.
[Context]
{retrieved_docs}

Question: {query}

'''

In [16]:
ai_message = llm.invoke(prompt)

In [17]:
ai_message.content

'중대성 평가는 조직이 환경, 사회, 지배구조(ESG) 측면에서 중요한 이슈를 식별하고 우선순위를 정하는 과정입니다. 이를 통해 조직은 경영활동과 이해관계자의 의사결정에 영향을 미치는 이슈를 파악하고, 이에 대한 대응 방안을 수립할 수 있습니다. 해양환경공단의 경우, 중대성 평가를 위해 내부 및 외부의 영향을 고려하였으며, 지속가능성 배경과 이해관계자 참여 등을 고려하여 중요한 이슈를 도출하였습니다.'